In [2]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 47.0 MB/s eta 0:00:00


In [3]:
from google.colab import files

# Upload your CSV file
uploaded = files.upload()

Saving satya.csv to satya.csv


In [4]:
import pandas as pd
import requests
from rdkit import Chem
from rdkit.Chem import AllChem
from google.colab import files
from google.colab import files
import os

In [5]:
saty = 'satya.csv'
vaty = pd.read_csv(saty)


In [6]:
vaty.head()

,Molecule ChEMBL ID,Smiles,Standard Type,Standard Value,mean_value,Class
0,CHEMBL3909538,CCC(CC)Oc1cc2c(cc1N1CCN(C3CCC3)CC1)C(C)(C)c1c(...,IC50,142.20,142.200000,1
1,CHEMBL3930736,CCc1cc2c(cc1N1CCC(O)(C3CC3)CC1)C(C)(C)c1[nH]c3...,IC50,14.20,14.200000,1
2,CHEMBL3916733,CC(C)C1(N2CCOCC2)CCN(c2cc3c(cn2)C(=O)c2c([nH]c...,IC50,9.37,9.370000,1
3,CHEMBL4753590,COc1cc(SCc2ccc(CN(C)C)o2)ccc1Nc1ncc(Cl)c(Nc2cc...,IC50,3.80,3.066667,1
4,CHEMBL4445283,COc1cc(NC(=O)NCc2cccnc2)ccc1Nc1ncc(Cl)c(Nc2ccc...,IC50,757.00,757.000000,1


In [7]:
import requests

def get_smiles_from_chembl(chembl_id):
    url = f"https://www.ebi.ac.uk/chembl/api/data/molecule/{chembl_id}.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get("molecule_structures", {}).get("canonical_smiles", None)
    else:
        return None


In [8]:
# Add SMILES to the DataFrame

if "ChEMBL_ID" in vaty.columns:
    vaty["SMILES"] = vaty["ChEMBL_ID"].apply(get_smiles_from_chembl)
    print(vaty.head())  # Verify the SMILES column

In [9]:
print(vaty)

    Molecule ChEMBL ID                                             Smiles  \
0        CHEMBL3909538  CCC(CC)Oc1cc2c(cc1N1CCN(C3CCC3)CC1)C(C)(C)c1c(...   
1        CHEMBL3930736  CCc1cc2c(cc1N1CCC(O)(C3CC3)CC1)C(C)(C)c1[nH]c3...   
2        CHEMBL3916733  CC(C)C1(N2CCOCC2)CCN(c2cc3c(cn2)C(=O)c2c([nH]c...   
3        CHEMBL4753590  COc1cc(SCc2ccc(CN(C)C)o2)ccc1Nc1ncc(Cl)c(Nc2cc...   
4        CHEMBL4445283  COc1cc(NC(=O)NCc2cccnc2)ccc1Nc1ncc(Cl)c(Nc2ccc...   
..                 ...                                                ...   
271      CHEMBL3943814    CCN(C)CCOc1ccc2c(c1)C(C)(C)C1Oc3cc(F)ccc3C1C2=O   
272      CHEMBL3972028  COc1cc(N2CCN(C(C)=O)CC2)ccc1Nc1ncc(Cl)c(Nc2ccc...   
273      CHEMBL3977731  CCNC(=O)N1CCN(c2ccc(Nc3nc(Nc4ccc(N5CCN(C(C)=O)...   
274      CHEMBL3926626  COc1cc(N2CCN(C(C)=O)CC2)ccc1Nc1ncc(C(F)(F)F)c(...   
275      CHEMBL3923931  CCCOc1cc(N2CCN(C(C)=O)CC2)ccc1Nc1nc(Nc2ccc(N3C...   

    Standard Type  Standard Value  mean_value  Class  
0            IC50   

In [10]:

# Create a directory to save the PDB files
output_dir = "/content/3d_structures"
os.makedirs(output_dir, exist_ok=True)

In [11]:
def smiles_to_3d(smiles, chembl_id, output_file):
    try:
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)  # Add hydrogens
        AllChem.EmbedMolecule(mol, AllChem.ETKDG())  # Generate 3D coordinates
        Chem.MolToPDBFile(mol, output_file)  # Save to PDB
        print(f"Saved {chembl_id} as {output_file}")
    except Exception as e:
        print(f"Error with SMILES {smiles}: {e}")


In [20]:
for index, row in vaty.iterrows():
  print(row)

Molecule ChEMBL ID                                        CHEMBL3909538
Smiles                CCC(CC)Oc1cc2c(cc1N1CCN(C3CCC3)CC1)C(C)(C)c1c(...
Standard Type                                                      IC50
Standard Value                                                    142.2
mean_value                                                        142.2
Class                                                                 1
Name: 0, dtype: object
Molecule ChEMBL ID                                        CHEMBL3930736
Smiles                CCc1cc2c(cc1N1CCC(O)(C3CC3)CC1)C(C)(C)c1[nH]c3...
Standard Type                                                      IC50
Standard Value                                                     14.2
mean_value                                                         14.2
Class                                                                 1
Name: 1, dtype: object
Molecule ChEMBL ID                                        CHEMBL3916733
Smiles            

In [27]:
for index, row in vaty.iterrows():
    if pd.notna(row["Smiles"]) and pd.notna(row["Molecule ChEMBL ID"]):  # Changed "ChEMBL_ID" to "Molecule ChEMBL ID"
        chembl_id = row["Molecule ChEMBL ID"]  # Changed "ChEMBL_ID" to "Molecule ChEMBL ID"
        output_file = f"{output_dir}/{chembl_id}_ligand_{index + 1}.pdb"  # Save with ChEMBL ID in the filename
        smiles_to_3d(row["Smiles"], chembl_id, output_file)

Saved CHEMBL3909538 as /content/3d_structures/CHEMBL3909538_ligand_1.pdb
Saved CHEMBL3930736 as /content/3d_structures/CHEMBL3930736_ligand_2.pdb
Saved CHEMBL3916733 as /content/3d_structures/CHEMBL3916733_ligand_3.pdb
Saved CHEMBL4753590 as /content/3d_structures/CHEMBL4753590_ligand_4.pdb
Saved CHEMBL4445283 as /content/3d_structures/CHEMBL4445283_ligand_5.pdb
Saved CHEMBL1202141 as /content/3d_structures/CHEMBL1202141_ligand_6.pdb
Saved CHEMBL1163566 as /content/3d_structures/CHEMBL1163566_ligand_7.pdb
Saved CHEMBL5284741 as /content/3d_structures/CHEMBL5284741_ligand_8.pdb
Saved CHEMBL2042829 as /content/3d_structures/CHEMBL2042829_ligand_9.pdb
Saved CHEMBL3786173 as /content/3d_structures/CHEMBL3786173_ligand_10.pdb
Saved CHEMBL3806112 as /content/3d_structures/CHEMBL3806112_ligand_11.pdb
Saved CHEMBL4218175 as /content/3d_structures/CHEMBL4218175_ligand_12.pdb
Saved CHEMBL4571241 as /content/3d_structures/CHEMBL4571241_ligand_13.pdb
Saved CHEMBL5177576 as /content/3d_structures/C

[10:01:10] UFFTYPER: Unrecognized atom type: S_5+4 (32)


Saved CHEMBL4588120 as /content/3d_structures/CHEMBL4588120_ligand_142.pdb
Saved CHEMBL2172313 as /content/3d_structures/CHEMBL2172313_ligand_143.pdb
Saved CHEMBL4446835 as /content/3d_structures/CHEMBL4446835_ligand_144.pdb
Saved CHEMBL3925372 as /content/3d_structures/CHEMBL3925372_ligand_145.pdb
Saved CHEMBL4469756 as /content/3d_structures/CHEMBL4469756_ligand_146.pdb
Saved CHEMBL2403843 as /content/3d_structures/CHEMBL2403843_ligand_147.pdb
Saved CHEMBL3128062 as /content/3d_structures/CHEMBL3128062_ligand_148.pdb
Saved CHEMBL4113131 as /content/3d_structures/CHEMBL4113131_ligand_149.pdb
Saved CHEMBL3608523 as /content/3d_structures/CHEMBL3608523_ligand_150.pdb
Saved CHEMBL3608522 as /content/3d_structures/CHEMBL3608522_ligand_151.pdb
Saved CHEMBL2403827 as /content/3d_structures/CHEMBL2403827_ligand_152.pdb
Saved CHEMBL4459025 as /content/3d_structures/CHEMBL4459025_ligand_153.pdb
Saved CHEMBL4519103 as /content/3d_structures/CHEMBL4519103_ligand_154.pdb
Saved CHEMBL4113977 as /c

In [35]:
from google.colab import files

In [41]:
import os
   for filename in os.listdir(content):
       if filename.endswith(".pdb"):  # Check for PDB files
           files.download(os.path.join(output_file, filename))

IndentationError: unexpected indent (<ipython-input-41-9c7e4ffffbc5>, line 2)

In [31]:
from google.colab import files
import os

# ... (your existing code) ...

# After the loop that generates PDB files:
for files in os.listdir():  # Loop through all files in the current directory
    if files.endswith(".pdb"):  # Check if the file is a PDB file
        files.download(files)

In [30]:
for filename in os.listdir():  # Loop through all files in the current directory
    if filename.endswith(".pdb"):  # Check if the file is a PDB file
        files.download(filename)

In [32]:
print(os.listdir(files))

['README.md', 'anscombe.json', 'mnist_train_small.csv', 'mnist_test.csv', 'california_housing_test.csv', 'california_housing_train.csv']


In [34]:
from google.colab import files

# List all files in the output directory
for filename in os.listdir(output_dir):
    if filename.endswith(".pdb"):
        files.download(f"{/content/3d_structures}/{CHEMBL3923931_ligand_276.pdb}")

SyntaxError: invalid decimal literal (<ipython-input-34-2a60d2ef40a8>, line 6)

In [42]:
import shutil
from google.colab import files

# Zip the directory containing PDB files
shutil.make_archive("/content/3d_structures", 'zip', output_dir)

# Download the zip file
files.download("/content/3d_structures.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [43]:
!apt-get install openbabel

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libinchi1 libmaeparser1 libopenbabel7
The following NEW packages will be installed:
  libinchi1 libmaeparser1 libopenbabel7 openbabel
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 3,903 kB of archives.
After this operation, 16.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libinchi1 amd64 1.03+dfsg-4 [455 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmaeparser1 amd64 1.2.4-1build1 [88.2 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopenbabel7 amd64 3.1.1+dfsg-6ubuntu5 [3,231 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 openbabel amd64 3.1.1+dfsg-6ubuntu5 [128 kB]
Fetched 3,903 kB in 0s (11.9 MB/s)
Selecting previously unselected package libinchi1.
(Reading database ... 124561 files and direc

In [48]:
import os

# Set the directory where your PDB files are stored
output_dir = "/content/3d_structures"  # Adjust this as needed


In [49]:
# Convert each PDB file in the directory to PDBQT
for filename in os.listdir(output_dir):
    if filename.endswith(".pdb"):
        input_file = os.path.join(output_dir, filename)
        output_file = os.path.join(output_dir, filename.replace(".pdb", ".pdbqt"))

        # Convert the PDB file to PDBQT using Open Babel
        !obabel $input_file -O $output_file

        print(f"Converted {filename} to {filename.replace('.pdb', '.pdbqt')}")

1 molecule converted
Converted CHEMBL5425607_ligand_204.pdb to CHEMBL5425607_ligand_204.pdbqt
1 molecule converted
Converted CHEMBL3822982_ligand_198.pdb to CHEMBL3822982_ligand_198.pdbqt
1 molecule converted
Converted CHEMBL3263993_ligand_63.pdb to CHEMBL3263993_ligand_63.pdbqt
1 molecule converted
Converted CHEMBL3918496_ligand_225.pdb to CHEMBL3918496_ligand_225.pdbqt
1 molecule converted
Converted CHEMBL1778719_ligand_24.pdb to CHEMBL1778719_ligand_24.pdbqt
1 molecule converted
Converted CHEMBL4753590_ligand_4.pdb to CHEMBL4753590_ligand_4.pdbqt
1 molecule converted
Converted CHEMBL2158528_ligand_20.pdb to CHEMBL2158528_ligand_20.pdbqt
1 molecule converted
Converted CHEMBL3694581_ligand_268.pdb to CHEMBL3694581_ligand_268.pdbqt
1 molecule converted
Converted CHEMBL5289155_ligand_21.pdb to CHEMBL5289155_ligand_21.pdbqt
1 molecule converted
Converted CHEMBL4114136_ligand_43.pdb to CHEMBL4114136_ligand_43.pdbqt
1 molecule converted
Converted CHEMBL3931692_ligand_100.pdb to CHEMBL39316

In [53]:
from google.colab import files

# Upload your files
uploaded = files.upload(/content/3d_structures/)

SyntaxError: invalid decimal literal (<ipython-input-53-90412cc1870d>, line 4)

In [54]:
output_dir = "/content/3d_structures"

In [55]:
from google.colab import files

# Loop through the files and download each PDBQT file
for filename in os.listdir(output_dir):
    if filename.endswith(".pdbqt"):
        files.download(os.path.join(output_dir, filename))
        print(f"Downloading {filename}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>